# Interactive Hyperdrive Tutorial


Agent0 provide a self contained Hyperdrive simulator using Interactive Hyperdrive. It works by managing a local Anvil
instance with an interactive and customizable Hyperdrive deployment, a data collection service, and a dashboard server
showing useful information for analysis.

To start, follow the installation instructions outlined [here](README.md) for interactive hyperdrive.


## Simulating trades using Interactive Hyperdrive

The following code initializes a local interactive hyperdrive with
(1) a local chain,
(2) a deployed hyperdrive pool on the local chain, and
(3) a funded agent attached to the pool ready to trade.


In [1]:
import datetime
from fixedpointmath import FixedPoint
from agent0.hyperdrive.interactive import ILocalHyperdrive, ILocalChain

chain = ILocalChain()
interactive_hyperdrive = ILocalHyperdrive(chain)
hyperdrive_agent0 = interactive_hyperdrive.init_agent(base=FixedPoint(100_000))

After initializing, we can use the `hyperdrive_agent0` object to simulate trades on the deployed pool.


In [2]:
open_long_event = hyperdrive_agent0.open_long(base=FixedPoint(100))
open_short_event = hyperdrive_agent0.open_short(bonds=FixedPoint(100))
add_liquidity_event = hyperdrive_agent0.add_liquidity(base=FixedPoint(100))

The output of these trades represents the corresponding emitted event from Hyperdrive. For example, the open long event
is shown below. See
[here](https://agent0.readthedocs.io/en/latest/autoapi/agent0/hyperdrive/interactive/event_types/index.html) for
documentation on the output event types.


In [3]:
open_long_event

OpenLong(trader='0x3201bDe1FA3Ac90FffF7bBD0b4F8f69dfBCd9c5b', asset_id=452312848583266388373324160190187140051835877600158453279131187532620414256, maturity_time=1709751600, base_amount=FixedPoint("100.0"), vault_share_amount=FixedPoint("99.999994292237768785"), as_base=True, bond_amount=FixedPoint("100.094931446539240406"))

We can use Interactive Hyperdrive to simulate advancing time and closing a position. For example, we can simulate
closing the long after a day.


In [4]:
# Advance time for a day
chain.advance_time(datetime.timedelta(days=1))
close_long_event = hyperdrive_agent0.close_long(
    maturity_time=open_long_event.maturity_time,
    bonds=open_long_event.bond_amount,
)

## Analyzing Hyperdrive

Interactive hyperdrive provides a fully managed data service that collects information from the chain, as
well as providing an interactive dashboard to view information on the pool. Running the dashboard within Interactive
Hyperdrive brings up a webpage that shows you basic information of the trades made on the pool, as well as information
on a specific agent, including the value of the portfolio.


In [5]:
interactive_hyperdrive.run_dashboard()

For customized analysis, Interactive Hyperdrive exposes the underlying data itself. See the `get_*` functions defined in
interactive hyperdrive
[here](https://agent0.readthedocs.io/en/latest/autoapi/agent0/hyperdrive/interactive/interactive_hyperdrive/index.html)
for more information.


## Trading policies

Agent0 introduces the concept of a policy, which defines trading behavior of an agent based
on a set of rules. Agent0 provides a set of policies ready to use out of the box. For example,
the code below details a policy that makes random trades.


In [6]:
from agent0.hyperdrive.policies import PolicyZoo

print(PolicyZoo.random.description())

  A simple demonstration agent that chooses its actions randomly.
  It can take 7 actions: open/close longs and shorts, add/remove liquidity, and redeem withdraw shares.
  Trade size is randomly drawn from a normal distribution with mean of 10% of budget and standard deviation of 1% of budget.
  A close action picks a random open position of the given type (long or short) and attempts to close its entire size.
  Withdrawals of liquidity and redemption of withdrawal shares is sized the same as an open position: N(0.1, 0.01) * budget.


We can attach a policy when initializing an agent and execute the underlying policy using
interactive hyperdrive. The output is then a list of output events corresponding with the
list of actions the policy took in that step. In the example below, we create a random agent
and execute one iteration of the policy, then printing out the trade the random agent made.


In [7]:
random_agent = interactive_hyperdrive.init_agent(
    base=FixedPoint(1_000),
    policy=PolicyZoo.random,
    policy_config=PolicyZoo.random.Config(rng_seed=123),
)
random_agent.execute_policy_action()

[OpenShort(trader='0xd5e58e7Cb404D367D4d7ADC0aA75Ce9E8B176EbA', asset_id=904625697166532776746648320380374280103671755200316906558262375063531163312, maturity_time=1709838000, base_amount=FixedPoint("0.165666458544196791"), vault_share_amount=FixedPoint("0.165643736102816493"), as_base=True, base_proceeds=FixedPoint("112.75489145076531133"), bond_amount=FixedPoint("112.879252612892491776"))]

See
[here](https://agent0.readthedocs.io/en/latest/autoapi/agent0/hyperdrive/policies/index.html#)
for a list and description of all implemented policies.


### Custom Policies

Agent0 also provides an interface to write a custom policy. In this tutorial, we'll write a
policy that simply opens a long if the fixed rate is above a threshold.
Similarly to existing policies, this policy can then be attached to interactive hyperdrive to
simulate the policy within the simulator.

We'll start by defining the class that subclasses from the base policy and define a custom policy
configuration that defines the threshold. We need to overwrite the `action` method in the
policy, which defines what actions the policy takes. In the example below, the agent will simply
open a single long for `open_long_amount` when the fixed rate reaches the `fixed_rate_threshold`,
and close the long at maturity.


In [8]:
# Relevant imports
from agent0.hyperdrive.policies import HyperdriveBasePolicy
from dataclasses import dataclass
from ethpy.hyperdrive import HyperdriveReadInterface
from agent0.hyperdrive import HyperdriveWallet
from agent0.hyperdrive.agent import open_long_trade, close_long_trade

In [9]:
class OpenLongPolicy(HyperdriveBasePolicy):
    @dataclass(kw_only=True)
    class Config(HyperdriveBasePolicy.Config):
        fixed_rate_threshold: FixedPoint
        open_long_amount: FixedPoint

    def action(self, interface: HyperdriveReadInterface, wallet: HyperdriveWallet):
        # Defines if the bot is done trading. We expect this bot to run continuously,
        # so this is always false.
        done_trading = False
        # If no longs in wallet, we check our fixed rate threshold and open the long if threshold reached.
        if len(wallet.longs) == 0:
            if interface.calc_fixed_rate() > self.config.fixed_rate_threshold:
                return [open_long_trade(self.config.open_long_amount)], done_trading
        # If there are longs in the wallet, we check for maturity and close them if maturity reached.
        else:
            for maturity_time, long in wallet.longs.items():
                if interface.get_block_timestamp(interface.get_current_block()) >= maturity_time:
                    return [close_long_trade(long.balance, maturity_time)], done_trading
        # We don't do any trades otherwise.
        return [], done_trading

### Running the trading policy in Interactive Hyperdrive

We can test our new policy using interactive hyperdrive before running it on an actual chain. We can do this by
attaching our new policy to a new agent, while passing in a fixed rate threshold of 6%.


In [10]:
policy_agent = interactive_hyperdrive.init_agent(
    base=FixedPoint(1_000_000),
    policy=OpenLongPolicy,
    policy_config=OpenLongPolicy.Config(
        fixed_rate_threshold=FixedPoint(0.06),
        open_long_amount=FixedPoint(100_000),
    ),
)

Lets take a look at the fixed rate after the set of trades we've made. We can see below that the current fixed rate sits
right around 5%.


In [11]:
interactive_hyperdrive.get_pool_state()[["block_number", "timestamp", "fixed_rate"]]

,block_number,timestamp,fixed_rate
0,20,2024-02-28 19:53:23,0.049999999999999996
1,21,2024-02-28 19:53:35,0.049999999999999996
2,22,2024-02-28 19:53:47,0.049999999999999996
3,23,2024-02-28 19:53:59,0.049999939182593864
4,24,2024-02-28 19:54:11,0.049999999941555074
5,25,2024-02-28 19:54:23,0.049999999941555074
6,73,2024-02-29 19:54:35,0.049999999941555074
7,74,2024-02-29 19:54:47,0.050000052064673868
8,75,2024-02-29 19:54:59,0.050000052064673868
9,76,2024-02-29 19:55:11,0.050000052064673868


We can now execute our new agent's policy. Since the fixed rate is below the threshold, we expect the agent to not make
any trades, denoted by the output of `execute_policy_action` being an empty list.


In [12]:
policy_agent.execute_policy_action()

[]

Let's make a trade to push the fixed rate to be 6%.


In [13]:
policy_agent.open_short(bonds=FixedPoint(20_000_000))
interactive_hyperdrive.get_pool_state()[["block_number", "timestamp", "fixed_rate"]]

,block_number,timestamp,fixed_rate
0,20,2024-02-28 19:53:23,0.049999999999999996
1,21,2024-02-28 19:53:35,0.049999999999999996
2,22,2024-02-28 19:53:47,0.049999999999999996
3,23,2024-02-28 19:53:59,0.049999939182593864
4,24,2024-02-28 19:54:11,0.049999999941555074
5,25,2024-02-28 19:54:23,0.049999999941555074
6,73,2024-02-29 19:54:35,0.049999999941555074
7,74,2024-02-29 19:54:47,0.050000052064673868
8,75,2024-02-29 19:54:59,0.050000052064673868
9,76,2024-02-29 19:55:11,0.050000052064673868


Once again, let's execute our policy and view the output. We can see that our policy opened a new long because the
fixed rate is above 6%.


In [14]:
policy_agent.execute_policy_action()

[OpenLong(trader='0xa3E519b323da5e3161661CF6e8fe3590b2939087', asset_id=452312848583266388373324160190187140051835877600158453279131187532620500656, maturity_time=1709838000, base_amount=FixedPoint("100000.0"), vault_share_amount=FixedPoint("99986.276614078975487391"), as_base=True, bond_amount=FixedPoint("100119.747042597701290677"))]

## Connecting to a remote chain

Along with a fully managed simulator, interactive hyperdrive also provides an interface for connecting to any existing remote chain and deployed hyperdrive pool. We can use this interface to make trades or execute any policy on the deployed hyperdrive pool.

In this tutorial, we'll use the Anvil node and Hyperdrive pool being hosted by the `ILocalChain`
and `ILocalHyperdrive` objects, but we can simply replace the corresponding configurations with any valid RPC and contract addresses.


In [15]:
from agent0.hyperdrive.interactive import IChain, IHyperdrive

# Get the RPC address and the hyperdrive contract addresses to connect to from the managed
# interactive hyperdrive objects.
rpc_uri = chain.rpc_uri
hyperdrive_addresses = interactive_hyperdrive.get_hyperdrive_addresses()

# We can specify other parameters as such.
# rpc_uri = "<rpc_uri>"
# TODO not all of these are strictly required, we may just need erc4626_hyperdrive.
# hyperdrive_addresses = IHyperdrive.Addresses(
#     base_token = "0x<base_token_contract_address>",
#     erc4626_hyperdrive = "0x<hyperdrive_contract_address>",
#     factory = "0x<hyperdrive_factory_contract_address>",
#     steth_hyperdrive="0x<steth_hyperdrive_contract_address>",
# )

# Connect to the remote chain and hyperdrive objects.
remote_chain = IChain(rpc_uri)
remote_hyperdrive = IHyperdrive(remote_chain, hyperdrive_addresses)

We now need to initialize an agent by providing a private key to a wallet. In this tutorial,
we'll simply use a random private key to initialize a new wallet, then fund the agent using
the provided `add_funds` method.

<div class="alert alert-block alert-info"> 
NOTE:: The `add_funds` method calls the `anvil_setBalance` to fund Eth and `mint` on the base
token contract to fund the wallet, which is only possible due to the remote chain and base token
contract supporting such calls. It's up to the user to ensure the wallet associated with the
private key is funded when running on a non-test chain.
</div>

<div class="alert alert-block alert-danger"> 
WARNING:: This tutorial generates a random private key and funds this wallet. In practice, the private
key should be passed into the script via an environment variable secret, and limit access
to the generated agent object to only the owner of the private key.
</div>

In addition, we also attach our custom agent as well to allow the agent to execute the policy we wrote eariler.


In [16]:
from agent0.base.make_key import make_private_key

remote_agent = remote_hyperdrive.init_agent(
    private_key=make_private_key(),
    policy=OpenLongPolicy,
    policy_config=OpenLongPolicy.Config(
        fixed_rate_threshold=FixedPoint(0.06),
        open_long_amount=FixedPoint(100_000),
    ),
)
# Add funds to the agent
remote_agent.add_funds(base=FixedPoint(1_000_000), eth=FixedPoint(100))
# Give the hyperdrive contract max approval to withdrawal funds from the remote agent
# TODO need to add setting a custom approval amount
remote_agent.set_max_approval()

In [17]:
remote_agent.wallet

HyperdriveWallet(address=HexBytes('0x3d94768938124747e471475d35d799cdc5fb01ff'), balance=Quantity(amount=FixedPoint("1000000.0"), unit=<TokenType.BASE: 'base'>), lp_tokens=FixedPoint("0.0"), withdraw_shares=FixedPoint("0.0"), longs={1709838000: Long(balance=FixedPoint("100119.747042597701290677"), maturity_time=1709838000)}, shorts={1709751600: Short(balance=FixedPoint("100.0"), maturity_time=1709751600), 1709838000: Short(balance=FixedPoint("20000112.879252612892491776"), maturity_time=1709838000)})

This agent is now able to make trades on the remote chain, as well as executing the underlying policy


In [ ]:
remote_agent.open_short(bonds=FixedPoint(200))

In [ ]:
remote_agent.execute_policy_action()

We can take a look at the agent's wallet, and we find that the wallet contains both the short we
opened manually, and the long we opened using the policy both exist in the wallet.


In [ ]:
remote_agent.wallet

## Cleanup

Recall that the local chain created at the beginning of this tutorial produces resources under
the hood that need to be cleaned up. The call below releases all the resources used by
interactive hyperdrive.


In [ ]:
chain.cleanup()